## 3절. 연습문제 - Oracle DB연동

In [45]:
## Oracle DB 연결

import cx_Oracle
import pandas as pd
import csv

oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521,sid='xe')
conn = cx_Oracle.connect('scott','tiger',dsn=oracle_dsn)
cursor = conn.cursor()

In [46]:
# 1. 입력

def insert_member():
    global conn
    cursor = conn.cursor()
    # 사용자로부터 이름, 전화, 이메일, 나이,등급(1~5), 기타특징 입력받아 DB에 추가한다
    name = input('이름:')
    phone = input('전화 번호:')
    email = input('이메일 주소:')
    age = int(input('나이:'))
    grade = int(input('등급:'))
    etc = input('기타사항:')
    
    cursor.execute("insert into member values (:name, :phone, :email, :age, :grade, :etc)",
                   {'name':name,'phone':phone, 'email':email,'age':age, 'grade':grade,'etc':etc})
    conn.commit()
    
    print('-----------------------------------')
    print('입력 완료')
    print('---------------------------------')

In [47]:
# 위의 메소드 test
insert_member()

이름:김세세
전화 번호:010-2231-1132
이메일 주소:sese@naver.com
나이:28
등급:4
기타사항:없음
-----------------------------------
입력 완료
---------------------------------


In [48]:
# 2. 전체조회 

def select_all():
    # SELECT * FROM MEMBER의 결과를 출력 (DataFrame 또는 list 형태 출력)
    cursor.execute('select * from member')
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    colnames = [col[0]  for col in cursor.description]
    df.columns = colnames
    print(df)

In [49]:
# 위의 메소드 test
select_all()

  NAME          PHONE           EMAIL  AGE  GRADE ETC
0  홍길동  010-2222-1111   HONG@HONG.COM   23      5  없음
1  김세세  010-2231-1132  sese@naver.com   28      4  없음
2  김기동  010-2222-1111    KIM@HONG.COM   33      5  없음
3    d              3               f    3      3   3


In [50]:
# 3. 이름 찾기 

def select_name():
    # 사용자로부터 검색하고자 하는 이름을 받아 결과 출력 
    name = input('검색하고자 하는 이름?')
    cursor.execute('select * from member where name=:name',{'name':name})
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    if len(result) ==0:
        print('---------------------------------')
        print('검색한 사람의 정보가 없습니다')
        print('---------------------------------')
    else:
        colnames = [op[0] for op in cursor.description]
        df.columns = colnames
    print(df)

In [51]:
# 위의 메소드 test

select_name()

검색하고자 하는 이름?김세세
  NAME          PHONE           EMAIL  AGE  GRADE ETC
0  김세세  010-2231-1132  sese@naver.com   28      4  없음


In [52]:
# 위의 메소드 test

select_name()

검색하고자 하는 이름?마마마
---------------------------------
검색한 사람의 정보가 없습니다
---------------------------------
Empty DataFrame
Columns: []
Index: []


In [53]:
# 4.메일 삭제 

def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 메일이 있는지 확인하고 
    # 해당 메일이 없다면 없다고 출력, 있으면 삭제 하고 삭제했다고 출력 
    global conn
    email = input('삭제할 자료의 이메일:')
    cursor.execute('select * from member where email=:email',{'email':email})
    result = cursor.fetchall()
    if len(result) == 0:
        print('---------------------------------')
        print('검색한 이메일이 존재하지 않습니다')
        print('---------------------------------')
    else:
        cursor.execute('delete from member where email=:email',{'email':email})
        print('---------------------------------')
        print('삭제완료')
        print('---------------------------------')
        conn.commit()

In [54]:
# 위의 메소드 test

delete_email()

삭제할 자료의 이메일:sese@naver.com
---------------------------------
삭제완료
---------------------------------


In [55]:
# 위의 메소드 test

delete_email()

삭제할 자료의 이메일:sese@naver.com
---------------------------------
검색한 이메일이 존재하지 않습니다
---------------------------------


In [56]:
# 5.CSV로 내보내기

def save_csv():
    #SELECT * FROM MEMBER의 결과를 CSV로 내보내기 (HEADER 포함)
    path = input('저장할 파일이름:')
    cursor.execute('select * from member')
    result = cursor.fetchall()
    result.insert(0,('name','phone','email','age','grade','etc')) 
    
    try:
        with open('data/%s'%path,'w',newline='') as f:
            writer = csv.writer(f)
            writer.writerows(result)
            print('--------------------------------')
            print('파일 저장 완료')
            print('---------------------------------')
    except FileNotFoundError:
        print('-------------------------------')
        print('해당 경로가 존재하지 않습니다')
        print('-------------------------------')


In [57]:
# 위의 메소드 test

save_csv()

저장할 파일이름:test.csv
--------------------------------
파일 저장 완료
---------------------------------


In [58]:
# 0.종료 -> main 함수에서 추가 
def close_sql():
    global conn
    conn.close()
    print('-------------------------------')
    print('종료합니다')
    print('---------------------------------')

In [59]:
def main():
    print('-------------------------------------------------------------------------')
    print('1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료',end=' ')
    print('\n-------------------------------------------------------------------------')
    enter = int(input('메뉴 선택:'))
    print('-------------------------------------------------------------------------')
    
    if enter ==1:
        try:
            insert_member()
        except ValueError:
            print('나이 또는 등급은 정수값을 입력하세요')
    elif enter==2:
        select_all()
    elif enter==3:
        select_name()
    elif enter ==4:
        delete_email()
    elif enter ==5:
        save_csv()
    elif enter ==0:
        close_sql()

In [61]:
# 1번:입력 메뉴 테스트 

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:1
-------------------------------------------------------------------------
이름:김시훈
전화 번호:010-3333-1111
이메일 주소:sihun@naver.com
나이:28
등급:4
기타사항:남자
-----------------------------------
입력 완료
---------------------------------


In [62]:
# 2번:전체 조회 메뉴 테스트 

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:2
-------------------------------------------------------------------------
  NAME          PHONE            EMAIL  AGE  GRADE ETC
0  홍길동  010-2222-1111    HONG@HONG.COM   23      5  없음
1  김시훈  010-3333-1111  sihun@naver.com   28      4  남자
2  김기동  010-2222-1111     KIM@HONG.COM   33      5  없음
3    d              3                f    3      3   3


In [64]:
# 3번:이름 찾기 메뉴 테스트 

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:3
-------------------------------------------------------------------------
검색하고자 하는 이름?김시훈
  NAME          PHONE            EMAIL  AGE  GRADE ETC
0  김시훈  010-3333-1111  sihun@naver.com   28      4  남자


In [65]:
# 4번: 메일 삭제 메뉴 테스트 

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:4
-------------------------------------------------------------------------
삭제할 자료의 이메일:sihun@naver.com
---------------------------------
삭제완료
---------------------------------


In [67]:
# 5번:csv 보내기 메뉴 테스트 

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:5
-------------------------------------------------------------------------
저장할 파일이름:고세현_Oracle.csv
--------------------------------
파일 저장 완료
---------------------------------


In [68]:
# 0번:종료하기 테스트

main()

-------------------------------------------------------------------------
1:입력|2:전체 조회|3:이름 찾기|4:메일 삭제|5:csv파일 출력|0:종료 
-------------------------------------------------------------------------
메뉴 선택:0
-------------------------------------------------------------------------
-------------------------------
종료합니다
---------------------------------
